### Imports

In [1]:
import os
import pickle
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Activation, Bidirectional, ConvLSTM2D, Attention, Dense, Flatten, MaxPool3D, MaxPool2D,BatchNormalization, Conv3D, GRU
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Model
from tensorflow.keras.backend import ctc_batch_cost, ctc_decode, ctc_label_dense_to_sparse, get_value
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input

import Levenshtein as Lev
import sys
from string import ascii_uppercase

In [2]:
print(tf.__version__)

2.0.0


### Paths

In [3]:
train_path = "./LibriSpeech100/train/train_all/"
dev_path = "./LibriSpeech100/dev/dev_all/"
test_path = "./LibriSpeech100/test/test_all/"

### Data Generator


In [4]:
class DataGenerator(Sequence):
    
    def __init__(self, path, to_fit = True):
        self.path = path
        self.list_X, self.list_Y = self.getLists()
        self.to_fit = to_fit
    
    
    def __len__(self):
        return len(self.list_X)
    
    
    def __getitem__(self, index):      
        dict_X = self.get_dict_X(index)   
        dict_Y = self.get_dict_Y(index)
        
        X, Y, input_len, label_len, y_strings = self.generate_XY(dict_X, dict_Y)
            
        return [X, y_strings, input_len, label_len], Y
    
    
    def getLists(self):
        list_X = []
        list_Y = []
        for item in sorted(os.listdir(self.path)):
            ext = item.split(".")[-1]
            if ext == 'pkl':
                list_X.append(item)
            elif ext == 'txt':
                list_Y.append(item)
        return list_X, list_Y
    
    
    def get_dict_X(self, index):
        file_name = self.path + self.list_X[index]
        with open(file_name, 'rb') as pickle_file:
            dict_X = pickle.load(pickle_file)
        return dict_X
    
    
    def get_dict_Y(self, index):
        filename = self.path + self.list_Y[index]
        file = open(filename)
        dict_Y = {}
        for line in file:
            data = line.split()
            key = data[0]
            value = ' '.join(data[1:])
            dict_Y[key] = value
        return dict_Y

    
    def generate_XY(self, dict_X, dict_Y):
        X = []
        Y = []
        Y_strings = []
        input_len = []
        label_len = []
        
        max_x = 0
        max_y = 0
        
        for key in dict_X:
            x_temp = dict_X[key]
            y_temp = dict_Y[key]
            if max_x < x_temp.shape[1]:
                max_x = x_temp.shape[1]
            if max_y < len(y_temp):
                max_y = len(y_temp)
        
        for key in dict_X:
            x_temp = dict_X[key]
            y_temp = dict_Y[key]
            Y_strings.append(y_temp)

            input_len.append(x_temp.shape[1])
            label_len.append(len(y_temp))
            
            to_pad_x = ( (0,0), (0, max_x - dict_X[key].shape[1]))
            to_pad_y = (  (0, max_y - len(dict_Y[key])))
            
            x_temp = np.pad(dict_X[key], pad_width = to_pad_x, mode='constant', constant_values=0)
            y_temp = self.generate_Y_array(dict_Y[key], max_y)
            X.append(x_temp.T)
            Y.append(y_temp)
          
        return np.stack(X), np.stack(Y), np.stack(input_len), np.stack(label_len), Y_strings

    
    def generate_Y_array(self, sentence, maxlen):
        space_token = ' '
        end_token = '>'
        blank_token = '%'
        apos_token = '\''
        while len(sentence) != maxlen:
            sentence += blank_token
        sentence += end_token
        
        alphabet = list(ascii_uppercase) + [space_token, apos_token, blank_token, end_token] 
        char_to_index = {}
        for idx, char in enumerate(alphabet):
            char_to_index[char] = idx

        y = []
        
        for char in sentence:
            y.append(char_to_index[char])
        
        return np.array(y)

### Create DataGenerator objects

In [5]:
train_data = DataGenerator(train_path)
val_data = DataGenerator(dev_path)
test_data = DataGenerator(test_path)

### Check Data loaded by the DataGenerator objects

In [6]:
x, y = train_data[0]
x, y_strings, input_len, label_len = x

In [7]:
print(x.shape)
print(input_len.shape)
print(label_len.shape)

print(len(y_strings))
print(y.shape)

(58, 727, 20)
(58,)
(58,)
58
(58, 319)


In [8]:
print(x.dtype)
print(y.dtype)
print(input_len.dtype)
print(label_len.dtype)

float32
int64
int64
int64


In [9]:
print(y_strings[0])

AND THAT IF SHE NOTICED ANYTHING ODD OR OUT OF PLACE SHE WOULD NEVER REST UNTIL SHE HAD FERRETED OUT THE WHYS AND WHEREFORES THEREOF THERE ARE PLENTY OF PEOPLE IN AVONLEA AND OUT OF IT WHO CAN ATTEND CLOSELY TO THEIR NEIGHBOR'S BUSINESS BY DINT OF NEGLECTING THEIR OWN


### Word Error Rate

In [10]:
def wer(s1, s2):

    s1 =s1.lower()
    s2 =s2.lower()
    b = set(s1.lower().split() + s2.lower().split())
    
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    return Lev.distance(''.join(w1), ''.join(w2))/float(len(s2.split()))

### Convert a tensor array to sentence

In [11]:
def indices_to_string(indices):
#     print(indices)
    space_token = ' '
    end_token = '>'
    blank_token = '%'
    apos_token = '\''
        
    alphabet = list(ascii_uppercase) + [space_token, apos_token, blank_token, end_token] 

    sentence = ''
    for idx in indices:
        sentence += alphabet[idx]
    
    return sentence

# Model Architecture

In [12]:
class BaseModel(Model):
    def __init__(self, op_dim = 30):
        super(BaseModel, self).__init__()
#         self.rnn = GRU(op_dim, return_sequences= True)
                self.rnn = GRU(200, return_sequences= True)
        #         self.batchnorm = BatchNormalization()
        #         self.time_dense = TimeDistributed(Dense(op_dim))
        
    def call(self, inputs):
        x = self.rnn(inputs)
        #         x = self.batchnorm(x)
        #         x = self.time_dense(x)
        return x

In [13]:
class ASRModel(Model):
    def __init__(self):
        super(ASRModel, self).__init__()
        self.base_model = BaseModel()
        self.activation = Activation('softmax')

    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.activation(x) 
        return x

### Build Model

In [14]:
model = ASRModel()
model.build(input_shape = (None, None, 20))
optimizer = tf.keras.optimizers.Adam()

### One Training Step

In [15]:
def train_one_step(model, optimizer, x, y_true, input_len, label_len, y_strings):
#     print('------------------------------')
#     print(x.shape)
#     print(y.shape)
#     print(input_len.shape)
#     print(label_len.shape)
    
    input_len = np.expand_dims(input_len, axis = 1)
    label_len = np.expand_dims(label_len, axis = 1)
#     print(input_len.shape)
#     print(label_len.shape)
            
    with tf.GradientTape() as tape:
        y_pred = model(x)
#         print(y_pred.shape)
        loss = ctc_batch_cost(y_true, y_pred, input_len, label_len)
    
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    input_len = np.squeeze(input_len)
    y_decode = ctc_decode(y_pred, input_len)[0][0]
    
#         print(y_decode)
#         print(len(y_strings))
    
    accuracy = 0.0
    
    for i in range(len(y_strings)):
        predicted_sentence = indices_to_string(y_decode[i])
#             print(predicted_sentence)
        accuracy += wer(predicted_sentence, y_strings[i])
            
    return tf.reduce_mean(loss), accuracy/len(y_strings)

### Training

In [16]:
def validate(model, x, y_true, input_len, label_len, y_strings, test = False):
    input_len = np.expand_dims(input_len, axis = 1)
    label_len = np.expand_dims(label_len, axis = 1)
    
    y_pred = model(x)
    loss = ctc_batch_cost(y_true, y_pred, input_len, label_len)
    
    input_len = np.squeeze(input_len)
    y_decode = ctc_decode(y_pred, input_len)[0][0]
    
    accuracy = 0.0
    
    for i in range(len(y_strings)):
        predicted_sentence = indices_to_string(y_decode[i])
#             print(predicted_sentence)
        accuracy += wer(predicted_sentence, y_strings[i])
        
        if test:
            print("Correct Sentence:", y_strings[i])
            print("Predicted Sentence:", predicted_sentence)
    
    return tf.reduce_mean(loss), accuracy/len(y_strings)    

In [17]:
def model_evaluate(model, val_ds, test = False):
    val_step = 0
    val_loss = 0.0
    val_accuracy = 0.0
            
    for inputs, y in val_ds:
        x, y_strings, ip_len, label_len = inputs
        val_step += 1       
        loss, accuracy = validate(model, x, y, ip_len, label_len, y_strings, test)
        val_loss += loss
        val_accuracy += accuracy
                
    val_loss /= val_step
    val_accuracy /= val_step

    tf.print(' Validation Loss:', val_loss, ' Validation WER: ', val_accuracy)
    
    return val_loss, val_accuracy

In [18]:
def model_fit(model, optimizer, train_ds, val_ds = None,epochs=5):
    
    losses = []
    accuracies = []
    val_losses = []
    val_acc = []
    
    for epoch in range(epochs):
        step = 0
        epoch_loss = 0.0
        epoch_accuracy = 0.0
        for inputs, y in train_ds:
            x, y_strings, ip_len, label_len = inputs
            step += 1
            loss, accuracy = train_one_step(model, optimizer, x, y, ip_len, label_len, y_strings)
            epoch_loss += loss
            epoch_accuracy += accuracy
            if step % 58 == 0:
                print(step)
                
            
        epoch_loss /= step
        epoch_accuracy /= step
        
        losses.append(epoch_loss)
        accuracies.append(epoch_accuracy)
        
        tf.print('Epoch: ', epoch+1, ' Loss:', epoch_loss, ' WER: ', epoch_accuracy)
        
        
        if val_ds:
            val_loss, val_accuracy = model_evaluate(model, val_ds)
            val_losses.append(val_loss)
            val_acc.append(val_accuracy)
        
                
    if not val_ds:    
        return losses, accuracies
    
    return losses, accuracies, val_losses, val_acc

In [19]:
losses, accuracies = model_fit(model, optimizer, train_data, val_ds = val_data)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
58
116
174
232
290
348
406
464
522
580
Epoch:  1  Loss: 1041.21619  WER:  0.9999990471562377
 Validation Loss: 593.47876  Validation WER:  1.0000896593408801
58
116
174
232
290
348
406
464
522
580
Epoch:  2  Loss: 957.412598  WER:  1.0761919898964039
 Validation Loss: 553.476929  Validation WER:  1.1917482641787707
58
116
174
232
290
348
406
464
522
580
Epoch:  3  Loss: 932.401184  WER:  1.1637782146462707
 Validation Loss: 552.769226  Validation WER:  1.2588789609659277
58
116
174
232
290
348
406
464
522
580
Epoch:  4  Loss: 932.130676  WER:  1.1998744751582282
 Validation Loss: 552.71167  Validation WER:  1.2947927478365717
58
116
174
232
290
348
406
464
522
580
Epoch:  5  Loss: 932.016907  WER:  1.227143441981416
 Validation Loss: 552.603638  Validation WER:  1.3077222630002474


ValueError: too many values to unpack (expected 2)

In [20]:
_, acc = model_evaluate(model, test_data, test=True)

Correct Sentence: AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS
Predicted Sentence:  E E E E O E T O E E E T O O E O T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: IN THE SILENCE THEIR DARK FIRE KINDLED THE DUSK INTO A TAWNY GLOW
Predicted Sentence:  E T O T E O E O T E O E O >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE RETREAT WILL BEGIN ON WEDNESDAY AFTERNOON IN HONOUR OF SAINT FRANCIS XAVIER WHOSE FEAST DAY IS SATURDAY
Predicted Sentence:  E T E E E O O O T E E E O O T E T T T E E T E T E E T E E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE CHAOS IN WHICH HIS ARDOUR EXTINGUISHED ITSELF WAS A COLD INDIFFERENT KNOWLEDGE OF HIMSELF
Predicted Sentence:  E E O T ET E O O E E T E T T O T O E E E O E E T O >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE RECTOR DID NOT ASK FO

Correct Sentence: FOR A FULL HOUR HE HAD PACED UP AND DOWN WAITING BUT HE COULD WAIT NO LONGER
Predicted Sentence:  O O E E T O E E E O >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE EUROPE THEY HAD COME FROM LAY OUT THERE BEYOND THE IRISH SEA EUROPE OF STRANGE TONGUES AND VALLEYED AND WOODBEGIRT AND CITADELLED AND OF ENTRENCHED AND MARSHALLED RACES
Predicted Sentence:  E E E E O O E E O E E O E O E E TE E E O TE T O T E O E E E E T T O T E O TE E T T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: PRIDE AFTER SATISFACTION UPLIFTED HIM LIKE LONG SLOW WAVES
Predicted Sentence:  O E E T T O E T E O E O T O E T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THEY WERE VOYAGING ACROSS THE DESERTS OF THE SKY A HOST OF NOMADS ON T

Correct Sentence: THE COLORIST SAYS FIRST OF ALL AS MY DELICIOUS PAROQUET WAS RUBY SO THIS NASTY VIPER SHALL BE BLACK AND THEN IS THE QUESTION CAN I ROUND HIM OFF EVEN THOUGH HE IS BLACK AND MAKE HIM SLIMY AND YET SPRINGY AND CLOSE DOWN CLOTTED LIKE A POOL OF BLACK BLOOD ON THE EARTH ALL THE SAME
Predicted Sentence:  E O O E T T T T O T O T T E E T E E T T E O E E E O T T O E E T O E T O E E T E E T O E O E O E T >>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE BROWN GROUND BENEATH IS LEFT FOR THE MOST PART ONE TOUCH OF BLACK IS PUT FOR THE HOLLOW TWO DELICATE LINES OF DARK GRAY DEFINE THE OUTER CURVE AND ONE LITTLE QUIVERING TOUCH OF WHITE DRAWS THE INNER EDGE OF THE MANDIBLE
Predicted Sentence:  E E T T O T O E E T O O E E O E O E O E O E O O T O E O E E E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THEY UNITE EVERY QUALITY AND SOMETIMES YOU WILL FIND ME REFERRING TO THEM AS COLORISTS SOMETIMES AS CHIAROSCURISTS
Predicted Sentence:  E E E E E O E T E O 

Correct Sentence: AY ME
Predicted Sentence:  E E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: NO MATTER THEN ALTHOUGH MY FOOT DID STAND UPON THE FARTHEST EARTH REMOV'D FROM THEE FOR NIMBLE THOUGHT CAN JUMP BOTH SEA AND LAND AS SOON AS THINK THE PLACE WHERE HE WOULD BE BUT AH
Predicted Sentence:  E T E E E E O O E T E T E E E T O T O O T E T E T E T O E E O T O T T E T TE E E E E TE E T T T E T E T E E  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: MY HEART DOTH PLEAD THAT THOU IN HIM DOST LIE A CLOSET NEVER PIERC'D WITH CRYSTAL EYES BUT THE DEFENDANT DOTH THAT PLEA DENY AND SAYS IN HIM THY FAIR APPEARANCE LIES
Predicted Sentence:  E O O T T E E O E T E E O T T O T E E E E T E T E T TET O E T E E T E E T O T T E E E E E T E TE E T E E T E E E T E T >>>>>>>>>>>>>>>>>>>>
Correct Sentence: THOUGHT KILLS ME THAT I AM NOT THOUGHT TO LEAP LARGE LENGTHS OF MILES WHEN THOU ART GONE BU

Correct Sentence: THE FIRST OF THESE TOUCHES CONVEYED THAT THE WRITTEN STATEMENT TOOK UP THE TALE AT A POINT AFTER IT HAD IN A MANNER BEGUN
Predicted Sentence:  T E T E E T T T E E E E T E T E T E T E E T T E E E E E E E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: HE QUITTED THE FIRE AND DROPPED BACK INTO HIS CHAIR
Predicted Sentence:  E E T E E O T E E E T E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE DEPARTING LADIES WHO HAD SAID THEY WOULD STAY DIDN'T OF COURSE THANK HEAVEN STAY THEY DEPARTED IN CONSEQUENCE OF ARRANGEMENTS MADE IN A RAGE OF CURIOSITY AS THEY PROFESSED PRODUCED BY THE TOUCHES WITH WHICH HE HAD ALREADY WORKED US UP
Predicted Sentence:  E E E E E E ET E E T E E E T E E T O E T T E E T E E E E E E E E T E T E E E T E T E E E E E E TE T E TE E T E E T T E ET E E T E T T E E O E E T 
Correct Sentence: HE HUNG FIR

...

Correct Sentence: I NEVER KNEW OF BUT ONE MAN WHO COULD EVER PLEASE HIM
Predicted Sentence:  E E T E E O E E E E ETE E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THE DOCTOR WHO ATTENDED THE INJURED CREATURE IN THIS CASE WAS SIMPLY TOLD THAT SHE SLIPPED AND FELL DOWN STAIRS AS SHE WAS COMING DOWN
Predicted Sentence:  E E E ET E E E ETE E T E TE E T E ET T ET TET E E E E TE ET E T E ETE E E E>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: THIS WAS WHAT DID THE MISCHIEF SO FAR AS THE RUNNING AWAY WAS CONCERNED
Predicted Sentence:  TE E T T E TE ETE T O ET E E E T E T E E>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: IT IS HARDLY NECESSARY TO SAY MORE OF THEM HERE
Predicted Sentence: E ET E TETETETE E E E E >>>>>>>>>>>>>>

Correct Sentence: OUR BAGGAGE WAS IMMEDIATELY CARRIED TO THE DECK OF THE FRIGATE I RUSHED ABOARD
Predicted Sentence:  T T E T T T T TE T E T E O ET >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: PLEASE FORGIVE ME FOR THIS UNDERHANDED WAY OF ADMITTING I HAD TURNED FORTY
Predicted Sentence:  T T T E E O E E T T T E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: CHAPTER THREE AS MASTER WISHES
Predicted Sentence:  T E T E T T E E T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: BUT WE'RE GOING JUST THE SAME
Predicted Sentence:  O T T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: NOT ONCE DID HE COMMENT ON THE LENGTH OR THE HARDSHIPS OF A JOURNEY
Predicted Sentence:  T T O T E T T E T E T E T>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: AND SO IF I'D BEEN DELAYED BY A QUARTER OF AN HOUR OR EVEN LESS THE FRIGAT

Correct Sentence: WHEN HE FINISHED SHE SAID CHEERFULLY
Predicted Sentence:  T T T T E T E T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: AT THE HEAD OF THE PINKIES WERE GHIP GHISIZZLE AND BUTTON BRIGHT WHO HAD THE PARROT ON HIS SHOULDER AND THEY WERE SUPPORTED BY CAPTAIN CORALIE AND CAPTAIN TINTINT AND ROSALIE THE WITCH
Predicted Sentence:  E E T E T E T E E E T T E E T E T T E T T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: SCUSE ME SAID TROT I NEGLECTED TO TELL YOU THAT YOU'RE NOT THE BOOLOOROO ANY MORE
Predicted Sentence:  T T ET E T E T E E E T T T E E E O E E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: WHEN FIRST THEY ENTERED THE THRONE ROOM THEY TRIED TO BE AS HAUGHTY AND SCORNFUL AS EVER BUT THE BLUES WHO WERE ASSEMBLED THERE ALL LAUGHED AT THEM AND JEERED THE

Correct Sentence: DESCEND O LITTLE CLOUD AND HOVER BEFORE THE EYES OF THEL
Predicted Sentence:  E E TE E E E O E E E E T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: WHICH THOU DOST SCATTER ON EVERY LITTLE BLADE OF GRASS THAT SPRINGS REVIVES THE MILKED COW AND TAMES THE FIRE BREATHING STEED
Predicted Sentence:  E T E T T E T E E E E E T E T E T E O E T T O E T T O E E E T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: BUT THEL IS LIKE A FAINT CLOUD KINDLED AT THE RISING SUN I VANISH FROM MY PEARLY THRONE AND WHO SHALL FIND MY PLACE
Predicted Sentence:  O E T O E E E E E E T T E E O E E O E E O O E E E T >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Correct Sentence: WHY AN EAR A WHIRLPOOL FIERCE TO DRAW CREATIONS IN
Predicted Sentence:  O E E O E T T E O E E T E >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: AND LAY ME DOWN IN THY COLD BED AND LEAVE MY SHINING LOT
Predicted Sentence:  E E O E E O E E E E E E E T E O >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: OR AN EYE OF GIFTS AND GRACES SHOWRING FRUITS AND COINED GOLD
Predicted Sentence:  OE O E E T E O E T E T E T O E E T E O E O >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Correct Sentence: NAY I RATHER THRILLED DIST

In [ ]:
print(acc)